In [120]:
import pandas as pd 
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import pickle 

from collections import Counter
from tqdm import tqdm

In [121]:
import time
start_time = time.time()

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

In [122]:
def clean_string_name(string):
    text = string.replace(' Size: 4 oz. * USDA Certified Organic Ingredient ** None remains after saponifying oils into soap and glycerin', '')
    pattern = "[\|\*\_\'\{}&]".format('"')
    regex1 = re.compile('\[.*]')
    regex2 = re.compile('\[.*}')
    text = re.sub(pattern, "", text)
    text = re.sub(regex1, "", text)
    text = re.sub(regex2, "", text)
    text = text.strip('\x0c')
    text = text.strip(' ')
    text = text.upper()
    text = text.replace('INACTIVE INGREDIENTS:', '')
    text = text.replace('ACTIVE INGREDIENTS:', '')

    return text

In [123]:
mismatch = ['CAVIAR WATER', 'STEM WATER']
def convert_new_ing_list(x):
    trimed = [name.strip(" '").strip("''") for name in x.strip('[]').split(',')]
    res = ''

    for i in range(len(trimed)):
        if i != (len(trimed) -1):
            if trimed[i] in mismatch:
                res = res + 'WATER' + ', '
            else:
                res = res + trimed[i] + ', '
        else: 
            if trimed[i] in mismatch:
                res = res + 'WATER' + ', '
            else:
                res = res + trimed[i]
    return res

# **Load dataset**

In [9]:
df_skin = pd.read_csv('../Database/EWG/ewg_skin_products_transformed_8208.csv')
df_sun = pd.read_csv('../Database/EWG/ewg_sun_products_transformed_1565.csv')
df_makeup = pd.read_csv('../Database/EWG/ewg_makeup_products_transformed_10527.csv')

df_hair = pd.read_csv('../Database/EWG/ewg_hair_products_transformed_4908.csv')
df_or = pd.read_csv('../Database/EWG/ewg_oralcare_products_transformed_937.csv')
df_frag = pd.read_csv('../Database/EWG/ewg_fragrance_products_transformed_484.csv')

In [10]:
df_all = pd.concat([df_skin, df_sun, df_makeup, df_hair, df_or, df_frag])

In [11]:
df_all.drop_duplicates(inplace=True)
df_all['new_ing_list_fixed'] = df_all['new_ing_list'].apply(convert_new_ing_list)
df_all['new_product_score'] = df_all['product_score'].map({'verified': 1,
                                                          '01': 2,
                                                          '02': 3,
                                                          '03': 4,
                                                          '04': 5,
                                                          '05': 6,
                                                          '06': 7,
                                                          '07': 8,
                                                          '08': 9,
                                                          '09': 10,
                                                          '10': 10,})
df_all.reset_index(drop=True, inplace=True)

In [12]:
drop_index = df_all[df_all['new_ing_list'] == '[]'].index.to_list()
df_all.drop(drop_index, inplace=True)
df_all.reset_index(drop=True, inplace=True)

In [13]:
df_all = df_all.sample(frac=1)
df_all.reset_index(drop=True, inplace=True)

In [124]:
df_dataset = df_all[df_all['new_product_score'] <=5]

In [125]:
df_dataset.reset_index(drop=True, inplace=True)

In [126]:
df_dataset.to_csv('recommendation_pool.csv', index=False)

In [127]:
with open('ingredient_idx.pickle', 'rb') as handle:
    ingredient_idx = pickle.load(handle)

In [128]:
len(ingredient_idx)

7433

In [129]:
corpus = []
for i in tqdm(range(len(df_dataset))):
    ingredients = df_dataset['new_ing_list_fixed'][i]
#     print(ingredients)
    tokens = ingredients.split(', ')    
    corpus.append(tokens)

100%|██████████| 17220/17220 [00:01<00:00, 9667.28it/s] 


In [130]:
# Define the oh_encoder function
def oh_encoder(tokens):
    x = np.zeros(N)    
    for ingredient in tokens:
        if ingredient in ingredient_idx.keys():
            idx = ingredient_idx[ingredient]
            x[idx] = 1
        else:
            pass
    return x

In [131]:
# Get the number of items and tokens 
M = len(df_dataset)                 # The number of the items
N = len(ingredient_idx)     # The number of the ingredients

# Initialize a matrix of zeros
A = np.zeros((M, N), dtype=np.uint8)

# Make a document-term matrix
i = 0
for tokens in corpus:
    A[i, :] = oh_encoder(tokens)
    i += 1

In [132]:
A.shape

(17220, 7433)

In [133]:
# columns name
ingredient_idx_key = [key for key in ingredient_idx.keys()]
df_sorted = pd.DataFrame(A, columns = ingredient_idx_key) 

# append to the original df
df_vector = pd.concat([df_dataset, df_sorted], axis=1)

In [134]:
df_vector

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img,ingredient_list,new_ing_list,new_ing_list_fixed,...,ALPINIA FORMOSANA SEED POWDER,SARGASSUM FUSIFORME EXTRACT,ISOSTEARAMIDOMORPHOLINE STEARATE,GLUTAMIC ACID FERMENT FILTRATE,APRICOT KERNEL AMINO ACIDS,OROTIC ACID,ACTINIDIA CHINENSIS FRUIT JUICE,CORN ACID,PEANUT ACID,TARAXACUM OFFICINALE
0,Hair,Baby Shampoo,https://www.ewg.org/skindeep/products/883715-3...,Whole Foods 365 Everyday Value,365 Everyday Value Baby Foaming Wash,04,https://phorcys-static.ewg.org/image/contents/...,"AQUA (WATER), GLYCERIN, PHENOXYETHANOL, CAPRYL...","['CAVIAR WATER', 'GLYCERIN', 'PHENOXYETHANOL',...","WATER, GLYCERIN, PHENOXYETHANOL, CAPRYLYL GLYC...",...,0,0,0,0,0,0,0,0,0,0
1,Skin,Exfoliant/Scrub,https://www.ewg.org/skindeep/products/789517-C...,C'est Moi,Vanilla Vibes Sugar Balm Body Scrub,verified,https://static.ewg.org/skindeep_images/7895/78...,"Sucrose, Glycine Soja (Soybean) Oil, Glycerin,...","['SUCROSE', 'GLYCINE SOJA OIL', 'GLYCERIN', 'S...","SUCROSE, GLYCINE SOJA OIL, GLYCERIN, SILICA SI...",...,0,0,0,0,0,0,0,0,0,0
2,Makeup,Eye Liner,https://www.ewg.org/skindeep/products/881867-C...,NaN,"CoverGirl Defining Moment All Day Eyeliner, Fs...",02,https://static.ewg.org/skindeep_images/8818/88...,"CYCLOPENTASILOXANE, TRIMETHYLSILOXYSILICATE, S...","['CYCLOPENTASILOXANE', 'TRIMETHYLSILOXYSILICAT...","CYCLOPENTASILOXANE, TRIMETHYLSILOXYSILICATE, S...",...,0,0,0,0,0,0,0,0,0,0
3,Hair,Shampoo,https://www.ewg.org/skindeep/products/927038-A...,AG,"Hair Care Balance Shampoo, Apple Cider Vinegar",02,https://static.ewg.org/skindeep_images/9270/92...,"Water (Aqua, Eau), Potassium Cocoyl Glycinate,...","['MARIS AQUA', 'LEAF)', 'POTASSIUM COCOYL GLYC...","MARIS AQUA, LEAF), POTASSIUM COCOYL GLYCINATE,...",...,0,0,0,0,0,0,0,0,0,0
4,Makeup,Brow Liner,https://www.ewg.org/skindeep/products/916405-M...,Milani,"Stay Put Brow Color, 04 Brunette",03,https://static.ewg.org/skindeep_images/9164/91...,"/INGR DIENTS/INGREDIENTES: ISODODECANE, DIMETH...","['DIMETHICONE', 'DICALCIUM PHOSPHATE', 'CYCLOP...","DIMETHICONE, DICALCIUM PHOSPHATE, CYCLOPENTASI...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17215,Makeup,Foundation,https://www.ewg.org/skindeep/products/837682-A...,Alima Pure,"Satin Matte Foundation, Netural 2",02,https://static.ewg.org/skindeep_images/8376/83...,"Mica (CI 77019), Titanium Dioxide (CI 77891), ...","['CI 77015', 'TITANIUM DIOXIDE', 'ZINC OXIDE',...","CI 77015, TITANIUM DIOXIDE, ZINC OXIDE, IRON O...",...,0,0,0,0,0,0,0,0,0,0
17216,Makeup,Bronzer/Highlighter,https://www.ewg.org/skindeep/products/927434-S...,Sol de Janeiro,"Glowmotions Glow Oil, Goldie Rocks",04,https://static.ewg.org/skindeep_images/9274/92...,"Isododecane, Hydrogenated Polyisobutene, Ethyl...","['ISODODECANE', 'HYDROGENATED POLYISOBUTENE', ...","ISODODECANE, HYDROGENATED POLYISOBUTENE, METHY...",...,0,0,0,0,0,0,0,0,0,0
17217,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/932919-E...,Earth Mama,"Mineral Sunscreen Lotion, SPF 40",01,https://static.ewg.org/skindeep_images/9329/93...,"Active Ingredients: Zinc Oxide 25%, Inactive I...","['ZINC OXIDE', 'COCOS NUCIFERA OIL', 'COCOS NU...","ZINC OXIDE, COCOS NUCIFERA OIL, COCOS NUCIFERA...",...,0,0,0,0,0,0,0,0,0,0
17218,Skin,Bath Oil/Salts/Soak,https://www.ewg.org/skindeep/products/779876-P...,Poofy Supernaturals,Energize Me Bath Salts,01,https://static.ewg.org/skindeep/img/ewg_missin...,"Dead Sea Salt , Epsom Salt, Pacific Sea Salt, ...","['SEA SALT', 'ISOMALT', 'SEA SALT', 'SEA SALT'...","SEA SALT, ISOMALT, SEA SALT, SEA SALT, SEA SAL...",...,0,0,0,0,0,0,0,0,0,0


# **KNN**

In [83]:
df_vector.iloc[:, 11:]

,WATER,PHENOXYETHANOL,GLYCERIN,IRON OXIDES,TITANIUM DIOXIDE,SILICA,TOCOPHERYL ACETATE,TOCOPHEROL,DIMETHICONE,CITRIC ACID,...,ALPINIA FORMOSANA SEED POWDER,SARGASSUM FUSIFORME EXTRACT,ISOSTEARAMIDOMORPHOLINE STEARATE,GLUTAMIC ACID FERMENT FILTRATE,APRICOT KERNEL AMINO ACIDS,OROTIC ACID,ACTINIDIA CHINENSIS FRUIT JUICE,CORN ACID,PEANUT ACID,TARAXACUM OFFICINALE
0,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12851,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12852,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12853,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
12854,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [135]:
y = df_vector['new_product_score'].to_numpy()
X = df_vector.iloc[:, 11:].to_numpy()

In [136]:
print('X_train shape: ', X.shape)
print('y_train shape: ', y.shape)

X_train shape:  (17220, 7433)
y_train shape:  (17220,)


In [137]:
from sklearn.neighbors import NearestNeighbors

In [138]:
nbrs = NearestNeighbors(n_neighbors=5).fit(X)

## **Testing**

In [89]:
df_test = pd.read_csv('ewg_all_products_vector.csv')

In [102]:
df_test[df_test['new_product_score'] == 10]

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img,ingredient_list,new_ing_list,new_ing_list_fixed,...,ALPINIA FORMOSANA SEED POWDER,SARGASSUM FUSIFORME EXTRACT,ISOSTEARAMIDOMORPHOLINE STEARATE,GLUTAMIC ACID FERMENT FILTRATE,APRICOT KERNEL AMINO ACIDS,OROTIC ACID,ACTINIDIA CHINENSIS FRUIT JUICE,CORN ACID,PEANUT ACID,TARAXACUM OFFICINALE
266,Fragrance,Fragrance For Men,https://www.ewg.org/skindeep/products/892978-A...,Acqua Di Parma,Colonia Essenza,09,https://static.ewg.org/skindeep/img/ewg_missin...,"Alcohol Denat, Water, Fragrance, Limonene, Hyd...","['ALCOHOL DENAT.', 'WATER', 'GRANITE', 'LIMONE...","ALCOHOL DENAT., WATER, GRANITE, LIMONENE, HYDR...",...,0,0,0,0,0,0,0,0,0,0
338,Hair,Hair Color And Bleaching,https://www.ewg.org/skindeep/products/877042-F...,Fright Night,"Colored Hair Spray, Cryptic Blue",10,https://static.ewg.org/skindeep/img/ewg_missin...,"ALCOHOL DENAT., HYDROFLUOROCARBON 152A, POLYAC...","['ALCOHOL DENAT.', 'HYDROFLUOROCARBON 152A', '...","ALCOHOL DENAT., HYDROFLUOROCARBON 152A, POLYAC...",...,0,0,0,0,0,0,0,0,0,0
562,Fragrance,Fragrance For Women,https://www.ewg.org/skindeep/products/882929-M...,Marc Jacobs,"Eau De Toilette Spray, Daisy Love",10,https://static.ewg.org/skindeep_images/8829/88...,"ALCOHOL DENAT., FRAGRANCE, WATER, ETHYLHEXYL M...","['ALCOHOL DENAT.', 'GRANITE', 'WATER', 'ETHYLH...","ALCOHOL DENAT., GRANITE, WATER, ETHYLHEXYL MET...",...,0,0,0,0,0,0,0,0,0,0
752,Fragrance,Fragrance For Men,https://www.ewg.org/skindeep/products/879553-D...,Dolce & Gabbana,The One Grey Eau De Toilette Intense for Men,09,https://static.ewg.org/skindeep_images/8795/87...,"ALCOHOL, PARFUM (FRAGRANCE), DIPROPYLENE GLYCO...","['ALCOHOL', 'DIPROPYLENE GLYCOL', 'CAVIAR WATE...","ALCOHOL, DIPROPYLENE GLYCOL, WATER, LIMONENE, ...",...,0,0,0,0,0,0,0,0,0,0
1052,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/928992-C...,Coppertone,"Sport Sunscreen Lotion, SPF 100",10,https://static.ewg.org/skindeep_images/9289/92...,"Active Ingredients: Avobenzone 3%, Homosalate ...","['AVOBENZONE', 'HOMOSALATE', 'OCTISALATE', 'OC...","AVOBENZONE, HOMOSALATE, OCTISALATE, OCTOCRYLEN...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21312,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/928918-B...,Banana Boat,"Kids MAX Clear Sunscreen Spray, SPF 100",10,https://static.ewg.org/skindeep_images/9289/92...,"Active Ingredients: Avobenzone 3.0%, Homosalat...","['AVOBENZONE', 'HOMOSALATE', 'OCTISALATE', 'OC...","AVOBENZONE, HOMOSALATE, OCTISALATE, OCTOCRYLEN...",...,0,0,0,0,0,0,0,0,0,0
21574,Fragrance,Fragrance For Women,https://www.ewg.org/skindeep/products/876763-G...,Gucci,Ladies Eau De Parfum Vaporisateur Natural Spra...,09,https://static.ewg.org/skindeep_images/8767/87...,"ALCOHOL DENAT., PARFUM (FRAGRANCE), AQUA (WATE...","['ALCOHOL DENAT.', 'CAVIAR WATER', 'ETHYLHEXYL...","ALCOHOL DENAT., WATER, ETHYLHEXYL METHOXYCINNA...",...,0,0,0,0,0,0,0,0,0,0
21747,Skin,Facial Moisturizer/Treatment,https://www.ewg.org/skindeep/products/900897-F...,Fourth Ray Beauty,Mellow Milk Mist,09,https://static.ewg.org/skindeep_images/9008/90...,"Water, Butylene Glycol, Glycerin, Propanediol,...","['WATER', 'BUTYLENE GLYCOL', 'GLYCERIN', 'PROP...","WATER, BUTYLENE GLYCOL, GLYCERIN, PROPANEDIOL,...",...,0,0,0,0,0,0,0,0,0,0
21873,Hair,Hair Color And Bleaching,https://www.ewg.org/skindeep/products/877073-F...,Fright Night,"Colored Hair Spray, Rotten Red",10,https://static.ewg.org/skindeep/img/ewg_missin...,"ALCOHOL DENAT., HYDROFLUOROCARBON 152A, POLYAC...","['ALCOHOL DENAT.', 'HYDROFLUOROCARBON 152A', '...","ALCOHOL DENAT., HYDROFLUOROCARBON 152A, POLYAC...",...,0,0,0,0,0,0,0,0,0,0


In [146]:
df_test.iloc[9830, :10]

cat_name                                                         Makeup
subcat_name                                               Facial Powder
product_url           https://www.ewg.org/skindeep/products/811657-B...
product_brand                                                   Benefit
product_name          The POREfessional Agent Zero Shine Shine-Vanis...
product_score                                                        06
product_img           https://static.ewg.org/skindeep_images/8116/81...
ingredient_list       Talc, Nylon-12, Aqua (Water), Magnesium Myrist...
new_ing_list          ['TALC', 'NYLON-12', 'CAVIAR WATER', 'MAGNESIU...
new_ing_list_fixed    TALC, NYLON-12, WATER, MAGNESIUM MYRISTATE, LA...
Name: 9830, dtype: object

In [147]:
test = df_test.iloc[9830, 11:].to_numpy(dtype= np.uint8)
test.shape

(7433,)

In [148]:
distances, indices = nbrs.kneighbors([test])
distances, indices

(array([[3.87298335, 4.        , 4.        , 4.        , 4.        ]]),
 array([[  592, 16665,  9619, 14174, 17070]]))

In [149]:
location = indices.tolist()[0]
location

[592, 16665, 9619, 14174, 17070]

In [150]:
df_dataset.iloc[location, :]

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img,ingredient_list,new_ing_list,new_ing_list_fixed,new_product_score
592,Makeup,Facial Powder,https://www.ewg.org/skindeep/products/639286-B...,Beautycounter,Mattifying Powder,verified,https://static.ewg.org/skindeep_images/6392/63...,"Zea Mays (Corn) Starch, Lauroyl Lysine, Silica...","['ZEA MAYS STARCH', 'LAUROYL LYSINE', 'SILICA'...","ZEA MAYS STARCH, LAUROYL LYSINE, SILICA, PHENO...",1
16665,Makeup,Facial Powder,https://www.ewg.org/skindeep/products/872195-J...,Juvia's Place,"I AM MAGIC Setting Powder, Gobi",04,https://static.ewg.org/skindeep_images/8721/87...,"Talc, Magnesium Myristate, Nylon-12, Zea Mays ...","['TALC', 'MAGNESIUM MYRISTATE', 'NYLON-12', 'Z...","TALC, MAGNESIUM MYRISTATE, NYLON-12, ZEA MAYS ...",5
9619,Makeup,Facial Powder,https://www.ewg.org/skindeep/products/815607-T...,tarte,"Shape Tape Setting Powder, Translucent",03,https://static.ewg.org/skindeep_images/8156/81...,"Talc, magnesium myristate, nylon-12, caprylic/...","['TALC', 'MAGNESIUM MYRISTATE', 'NYLON-12', 'C...","TALC, MAGNESIUM MYRISTATE, NYLON-12, CAPRIC TR...",4
14174,Makeup,Facial Powder,https://www.ewg.org/skindeep/products/872196-J...,Juvia's Place,"I AM MAGIC Setting Powder, Kalahari",04,https://static.ewg.org/skindeep_images/8721/87...,"Talc, Magnesium Myristate, Nylon-12, Zea Mays ...","['TALC', 'MAGNESIUM MYRISTATE', 'NYLON-12', 'Z...","TALC, MAGNESIUM MYRISTATE, NYLON-12, ZEA MAYS ...",5
17070,Makeup,Facial Powder,https://www.ewg.org/skindeep/products/872193-J...,Juvia's Place,"I AM MAGIC Setting Powder, Namib",04,https://static.ewg.org/skindeep_images/8721/87...,"Talc, Magnesium Myristate, Nylon-12, Zea Mays ...","['TALC', 'MAGNESIUM MYRISTATE', 'NYLON-12', 'Z...","TALC, MAGNESIUM MYRISTATE, NYLON-12, ZEA MAYS ...",5


In [151]:
import pickle

pickle.dump(nbrs, open('recommendation.pkl', 'wb'))

In [152]:
# Your code here
with open('recommendation.pkl', 'rb') as model:
    reload_rf = pickle.load(model)

distances, indices = reload_rf.kneighbors([test])
distances, indices 

(array([[3.87298335, 4.        , 4.        , 4.        , 4.        ]]),
 array([[  592, 16665,  9619, 14174, 17070]]))

# **Model**

In [28]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import NearestNeighbors

class IngredientTokenizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X): # X: nested list
        M = len(X)
        N = len(ingredient_idx)
        A = np.zeros((M, N), dtype=np.uint8)
        i= 0
        for ing_list in X:
            x = np.zeros(N, dtype=np.uint8)
            for ingredient in ing_list:
                # Get the index for each ingredient
                if ingredient in ingredient_idx.keys():
                    idx = ingredient_idx[ingredient]
                    x[idx] = 1
                else:
                    pass
            
            A[i, :] = x
            i += 1
        return A

In [26]:
def column_to_list(df, col_name):
    corpus = []
    for i in tqdm(range(len(df))):
        
        ingredients = df[col_name][i]
        if not isinstance(ingredients, str):
            print(ingredients)
            ingredients = str(ingredients)
        tokens = ingredients.split(', ')    
        corpus.append(tokens)
    return corpus

In [24]:
X = column_to_list(df_all, 'new_ing_list_fixed')

100%|██████████| 23260/23260 [00:02<00:00, 7854.31it/s] 


In [29]:
knn_pipeline = Pipeline([
        ('tokenizer', IngredientTokenizer()),
        ('KNN', NearestNeighbors(n_neighbors=5)),
    ])

In [30]:
knn_pipeline.fit(X)

Pipeline(steps=[('tokenizer', IngredientTokenizer()),
                ('KNN', NearestNeighbors())])

In [37]:
df_all.loc[90, :]

cat_name                                                            Sun
subcat_name                                        Moisturizer With SPF
product_url           https://www.ewg.org/skindeep/products/932934-F...
product_brand                                          First Aid Beauty
product_name               Ultra Repair Tinted Moisturizer, Tan, SPF 30
product_score                                                        03
product_img           https://static.ewg.org/skindeep_images/9329/93...
ingredient_list       Active Ingredients: Avobenzone 3%, Octinoxate ...
new_ing_list          ['AVOBENZONE', 'OCTINOXATE', 'OCTISALATE', 'OC...
new_ing_list_fixed    AVOBENZONE, OCTINOXATE, OCTISALATE, OCTOCRYLEN...
Name: 90, dtype: object

In [41]:
ext = df_all.iloc[90]['new_ing_list_fixed']
test = [[i for i in ext.split(' ')]]
test

[['AVOBENZONE,',
  'OCTINOXATE,',
  'OCTISALATE,',
  'OCTOCRYLENE,',
  'CYCLOPENTASILOXANE,',
  'DIMETHICONE,',
  'CETEARYL',
  'ALCOHOL,',
  'CETEARYL',
  'GLUCOSIDE,',
  'POTASSIUM',
  'CETYL',
  'PHOSPHATE,',
  'POLYGLYCERYL-4',
  'ISOSTEARATE,',
  'DIMETHICONE',
  'CROSSPOLYMER,',
  'TOCOPHERYL',
  'ACETATE,',
  'GLYCERIN,',
  'CETYL',
  'PEG-8',
  'DIMETHICONE,',
  'CETYL',
  'DIMETHICONE,',
  'POLYSORBATE',
  '60,',
  'BUTYLENE',
  'GLYCOL,',
  'HEXYL',
  'LAURATE,',
  'COLLOIDAL',
  'OATMEAL,',
  'PHENOXYETHANOL,',
  'SODIUM',
  'DEHYDROACETATE,',
  'SODIUM',
  'HYALURONATE,',
  'CAMELLIA',
  'SINENSIS',
  'LEAF',
  'EXTRACT,',
  'CARBOMER,',
  'TRIBEHENIN,',
  'LIMONENE,',
  'TETRASODIUM',
  'EDTA,',
  'AMINOMETHYL',
  'PROPANOL,',
  'CITRUS',
  'NOBILIS',
  'PEEL',
  'OIL,',
  'ALLANTOIN,',
  'SODIUM',
  'ASCORBATE,',
  'GLYCYRRHIZA',
  'GLABRA',
  'ROOT',
  'EXTRACT,',
  'CHRYSANTHEMUM',
  'PARTHENIUM',
  'FLOWER',
  'EXTRACT,',
  'MICA']]

In [139]:
distances, indices = knn_pipeline.kneighbors([test])
distances, indices

NameError: name 'knn_pipeline' is not defined

In [ ]:
df_all.iloc[90,:]

In [ ]:
location = indices.tolist()[0]
df_all.iloc[location, :]